## SF crime data analysis and modeling

### In this notebook, we will use Spark SQL for big data analysis on SF crime data. (https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry). 
The first part of project is OLAP for scrime data analysis 
The second part is unsupervised learning for spatial data analysis 
The option part is the time series data analysis

In [3]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"


In [4]:
data_path = "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv"
# use this file name later

In [5]:
#Get the DataFrame
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df_opt1 = spark.read.format("csv").option("header", "true").load(data_path)
display(df_opt1)
df_opt1.createOrReplaceTempView("sf_crime")



IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,DELETE - Fire Prevention Districts,DELETE - Police Districts,DELETE - Supervisor Districts,DELETE - Zip Codes,DELETE - Neighborhoods,DELETE - 2017 Fix It Zones,Civic Center Harm Reduction Project Boundary,Fix It Zones as of 2017-11-06,DELETE - HSOC Zones,Fix It Zones as of 2018-02-07,"CBD, BID and GBD Boundaries as of 2017","Areas of Vulnerability, 2016",Central Market/Tenderloin Boundary,Central Market/Tenderloin Boundary Polygon - Updated,HSOC Zones as of 2018-06-05,OWED Public Spaces
146196161,NON-CRIMINAL,LOST PROPERTY,Tuesday,09/23/2014,01:00,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403404791479,37.775420706711,POINT (-122.403404791479 37.775420706711),14619616171000,32,1,10,34,14,2,9,28853,34,null,null,null,null,null,null,2,null,null,null,null
150045675,ASSAULT,BATTERY,Thursday,01/15/2015,17:00,TARAVAL,NONE,1800 Block of VICENTE ST,-122.48560378101,37.7388214326705,POINT (-122.48560378101 37.7388214326705),15004567504134,40,10,7,35,1,8,3,29491,35,null,null,null,null,null,null,1,null,null,null,null
140632022,SUSPICIOUS OCC,INVESTIGATIVE DETENTION,Wednesday,07/30/2014,09:32,BAYVIEW,NONE,100 Block of GILLETTE AV,-122.396535107224,37.7106603302503,POINT (-122.396535107224 37.7106603302503),14063202264085,89,2,9,1,10,3,8,309,1,null,null,null,null,null,null,1,null,null,null,null
150383259,ASSAULT,BATTERY,Saturday,05/02/2015,23:10,BAYVIEW,"ARREST, BOOKED",2400 Block of PHELPS ST,-122.400130573297,37.7300925390327,POINT (-122.400130573297 37.7300925390327),15038325904134,87,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null
040753980,OTHER OFFENSES,RECKLESS DRIVING,Friday,07/02/2004,13:43,BAYVIEW,NONE,I-280 / CESAR CHAVEZ ST,-120.5,90,POINT (-120.5 90),4075398065020,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
040855122,SUICIDE,SUICIDE BY JUMPING,Tuesday,07/27/2004,15:19,SOUTHERN,NONE,500 Block of I-80,-122.386667033903,37.7898821569191,POINT (-122.386667033903 37.7898821569191),4085512260170,null,null,null,null,null,null,null,28856,null,null,null,null,null,null,null,null,null,null,null,null
066085191,NON-CRIMINAL,LOST PROPERTY,Sunday,11/19/2006,17:45,BAYVIEW,NONE,0 Block of GIANTS DR,-122.38750147945,37.716878646429,POINT (-122.38750147945 37.716878646429),6608519171000,88,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null
050908404,VEHICLE THEFT,STOLEN AUTOMOBILE,Saturday,08/13/2005,17:00,TENDERLOIN,NONE,JENNINGS CT / INGALLS ST,-120.5,90,POINT (-120.5 90),5090840407021,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
090768064,ARSON,ARSON OF A VEHICLE,Tuesday,07/28/2009,23:26,BAYVIEW,NONE,SELBY ST / OAKDALE AV,-122.399686082806,37.739901780585,POINT (-122.399686082806 37.739901780585),9076806426031,87,2,9,1,10,3,8,58,1,null,null,null,null,null,null,2,null,null,null,null
111027676,ASSAULT,BATTERY,Saturday,12/24/2011,07:00,SOUTHERN,NONE,0 Block of DORE ST,-122.412933062384,37.7739274524819,POINT (-122.412933062384 37.7739274524819),11102767604134,32,1,10,34,8,2,9,28853,34,null,1,null,1,null,null,2,null,null,1,null


#### (OLAP)Counts The Number of Crimes for Different Category.

In [7]:
q1_result = df_opt1.groupBy('category').count().orderBy('count', ascending=False)
display(q1_result)

category,count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


In [8]:
#Spark SQL based
crimeCategory = spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC")
display(crimeCategory)

category,Count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


In [9]:
crimes_pd_df = crimeCategory.toPandas()

####(OLAP)Counts The Number of Crimes For Different District, and Visualize the Results

In [11]:

## use sql to compute the statistic result
crimeDistrict = spark.sql("SELECT PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY 1 ORDER BY Count DESC")
display(crimeDistrict)

## visualize the results
crimes_pd_df_1 = crimeDistrict.toPandas()


PdDistrict,Count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


#### (OLAP)Count The Number of Crimes Each "Sunday" at "SF downtown".

####Note:
SF downtown is defiend via the range of spatial location. For example, you can use a rectangle to define the SF downtown, or you can define a cicle with center as well. Thus, we write our own UDF function to filter data which are located inside certain spatial range.

In [14]:
#Using a udf to find all the crimes in down town SF.
from pyspark.sql.types import BooleanType

def within_geo_coordinates(x_in, y_in):
  if float(x_in) > -122.4313 and float(x_in) < -122.4213 and float(y_in) > 37.7540 and float(y_in) < 37.7740:
    return True
  else:
    return False
  
within_geo_coordinates_udf = udf(within_geo_coordinates, BooleanType())#register udf
df_opt1.withColumn('in_downTownSF', within_geo_coordinates_udf("X", "Y"))
df_q3=df_opt1.select("IncidntNum", "Category","Date", "DayOfWeek", within_geo_coordinates_udf("X", "Y").alias("in_downTownSF")).orderBy("in_downTownSF", ascending = False)
display(df_q3)



IncidntNum,Category,Date,DayOfWeek,in_downTownSF
160433361,LARCENY/THEFT,05/26/2016,Thursday,true
150506138,OTHER OFFENSES,06/10/2015,Wednesday,true
186042712,LARCENY/THEFT,02/11/2018,Sunday,true
170302405,NON-CRIMINAL,04/13/2017,Thursday,true
171012316,OTHER OFFENSES,12/15/2017,Friday,true
170127980,OTHER OFFENSES,02/10/2017,Friday,true
176043930,LARCENY/THEFT,02/14/2017,Tuesday,true
160433452,ASSAULT,05/27/2016,Friday,true
171046791,LARCENY/THEFT,12/28/2017,Thursday,true
186041037,NON-CRIMINAL,02/14/2018,Wednesday,true


In [15]:
#count crime number of each sunday

df_q3.createOrReplaceTempView("sf_crime_q3")
crimeDowntown = spark.sql("SELECT Date, Count(*) AS Count FROM sf_crime_q3 WHERE DayOfWeek = 'Sunday' GROUP By Date Order By Date ASC")
display(crimeDowntown)

Date,Count
01/01/2006,621
01/01/2012,530
01/01/2017,485
01/02/2005,304
01/02/2011,305
01/03/2010,356
01/03/2016,316
01/04/2004,351
01/04/2009,350
01/04/2015,335


#### (OLAP)Analysis The Number of Crime in Each Month of 2015, 2016, 2017, 2018.

In [17]:
#%sql select substring(Date,7) as Year, substring(Date,1,2) as Month, count(*)  from sf_crime
#group by Year, Month
#having Year Between 2015 and 2018
#order by Year, Month
df_opt1.createOrReplaceTempView("sf_crime")
temp = spark.sql("select substring(Date,7) as Year, substring(Date,1,2) as Month, count(*)  from sf_crime group by Year, Month having Year Between 2015 and 2018 order by Year, Month")
display(temp)


Year,Month,count(1)
2015,01,13606
2015,02,12329
2015,03,13929
2015,04,12959
2015,05,13729
2015,06,13304
2015,07,13365
2015,08,13730
2015,09,12896
2015,10,13147


In [18]:
df_opt1.createOrReplaceTempView("sf_crime")
temp = spark.sql("select substring(Date,7) as Year, substring(Date,1,2) as Month, count(*)  from sf_crime group by Year, Month having Year Between 2015 and 2017 order by Year, Month")
display(temp)

Year,Month,count(1)
2015,01,13606
2015,02,12329
2015,03,13929
2015,04,12959
2015,05,13729
2015,06,13304
2015,07,13365
2015,08,13730
2015,09,12896
2015,10,13147


####Note (Insight)
1. Because 2018 has a significant different pattern from the previous years and missed almost half year's data, we viewed it as an exception and exclude it from the analysis. 
2. The graph shows a cycling pattern with higher crime rate months immediately follow by lower crime rate months. This fluctuation can be caused by the intensity of police enforcement follows the crime rate. When the crime rate is high, the police enforcement intensifies. Then the crime rate drops. The enforcement relaxes after the drops which then follow by a higher crime rate. To smooth the crime curve over months, law enforcement can place the resource more evenly in the future. And police can be more alert and prepared for the high crime month even in a low crime month
3. Based on the data from 2015, 2016 and 2017, San Francisco has similar monthly crime pattern in different years. The crime rate always decreases significantly from January to February and bounce back from February to March. The number of crimes also falls from October to the end of the year. Due to the lower crime rate in each year's February and October, event holders and business owners may consider scheduling their events and visits in these two months with lower crime rate. 
4. Based on the data, the January and March usually have a higher crime rate relatively. It can be caused by the concentration of holidays in these two months. For example, in each year, SF will hold a parade for St.Patrick day in March. Due to the concentration of holidays in January and March, the number of tourists increases which increase the crime rate. 
5. The charts also show a decrease of crime rate after October. From some crime study, temperature can affect people's aggressive behavior. The lower temperature in winter days in SF can be the reason behind the lower crime rate

#### (OLAP)Analysis The Number of Crime w.r.t The Hour in December 15 Each Year

In [21]:
#Change the date format of the date column
from pyspark.sql.functions import *
df_update = df_opt1.withColumn("Date", to_date(col("Date"), "MM/dd/yyyy")) ##change datetype from string to date
df_update.createOrReplaceTempView("sf_crime")

temp = spark.sql("Select Date, hour(Time) As Hour, Count(*) as Count From sf_crime Group By Date, Hour Having Year(Date) between 2015 and 2017 and Month(Date)=12 and DayOfMonth(Date)=15 Order By 1,2")
display(temp)

Date,Hour,Count
2015-12-15,0,15
2015-12-15,1,6
2015-12-15,2,5
2015-12-15,3,4
2015-12-15,4,10
2015-12-15,5,3
2015-12-15,6,4
2015-12-15,7,8
2015-12-15,8,12
2015-12-15,9,10


1. We first calculated the crime numbers in every hour for 12/15/2015, 12/15/2016 and 12/15/2017. In order to see the pattern more clearly, we stacked the lines. From the line chart, the crime number decreased from mid night and hit the bottom before sunrise. After sunrise, the crime rate begins to increase. 
2.  In order to know if this pattern is only a coincident for December 15, we then accumulated all the crime numbers each hours for all the dates and have the below line chart.

In [23]:
temp = spark.sql("Select Date, hour(Time) As Hour, Count(*) as Count From sf_crime Group By Date, Hour Order By 1,2")
display(temp)

Date,Hour,Count
2003-01-01,0,103
2003-01-01,1,31
2003-01-01,2,38
2003-01-01,3,17
2003-01-01,4,5
2003-01-01,5,7
2003-01-01,6,10
2003-01-01,7,21
2003-01-01,8,18
2003-01-01,9,25


#### Note(Insight)
1. The graph shows a U shape from mid night to noon. 
2. Based on our crime data, san Francisco has lower crime numbers in the morning especially before the sunrise compared to the rest of the day. Morning is safer than afternoon. We will suggest traveler schedule their visit in the morning around 5am to 8am of a day.

#### (OLAP)Find out The Top-3 Danger District  
1. find out the crime event w.r.t category and time (hour) from the results. 
2. give your advice to distribute the police based on the analysis results.

In [26]:
#Find out the top-3 danger district
crimeDistrictTop3 = spark.sql("SELECT PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY 1 ORDER BY Count DESC LIMIT 3")
display(crimeDistrictTop3)

PdDistrict,Count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713


Based on the number of crime in each district, we found that the three most dangerours district in SF are Southern, Mission and Northern.

In [28]:
#find out the crime event w.r.t category and time (hour) from the result of step 1
#First of all, we pull out  the crime event data w.r.t category and time for Southern District.
crimeDistrictSouthern = spark.sql("SELECT PdDistrict, Category, hour(Time) As Hour, COUNT(*) AS Count FROM sf_crime GROUP BY 1,2,3 having PdDistrict = 'SOUTHERN' ORDER BY 1, 2,3 ASC ")
display(crimeDistrictSouthern)

PdDistrict,Category,Hour,Count
SOUTHERN,ARSON,0,31
SOUTHERN,ARSON,1,27
SOUTHERN,ARSON,2,25
SOUTHERN,ARSON,3,21
SOUTHERN,ARSON,4,20
SOUTHERN,ARSON,5,20
SOUTHERN,ARSON,6,14
SOUTHERN,ARSON,7,9
SOUTHERN,ARSON,8,17
SOUTHERN,ARSON,9,7


#### Note(Insight)
1. Crimes at all categories tend to follow the rule we previously discovered for SF city, which is the crime number decrease from mid night and increase after sunrise. That means Southern District is safer at night before sun rise and in the morning before noon.
2. Based on the line chart depicted by calculating the crime number distribution during a day respect to different categories, we found that the most common crime in Southern District is theft and these type of crime events concentrated at night. The crime in this category peaks between 18 and 19 o'clock.
3. The second and the third most serious crimes in this district are Assault and Drug. The Assault category has a smooth distribution which means the crime happened evenly at each hour during the day. 
4. The Drug category crime increase during the day and decrease after sunset.

In [30]:
#Secondly, we pull out  the crime event data w.r.t category and time for Mission District.
crimeDistrictMission = spark.sql("SELECT PdDistrict, Category, hour(Time) As Hour, COUNT(*) AS Count FROM sf_crime GROUP BY 1,2,3 having PdDistrict = 'MISSION' ORDER BY 1, 2,3 ASC ")
display(crimeDistrictMission)

PdDistrict,Category,Hour,Count
MISSION,ARSON,0,35
MISSION,ARSON,1,23
MISSION,ARSON,2,28
MISSION,ARSON,3,36
MISSION,ARSON,4,27
MISSION,ARSON,5,26
MISSION,ARSON,6,12
MISSION,ARSON,7,9
MISSION,ARSON,8,7
MISSION,ARSON,9,14


In [31]:
#Finally, we pull out  the crime event data w.r.t category and time for Mission District.
crimeDistrictNorthern = spark.sql("SELECT PdDistrict, Category, hour(Time) As Hour, COUNT(*) AS Count FROM sf_crime GROUP BY 1,2,3 having PdDistrict = 'NORTHERN' ORDER BY 1, 2,3 ASC ")
display(crimeDistrictNorthern)

PdDistrict,Category,Hour,Count
NORTHERN,ARSON,0,18
NORTHERN,ARSON,1,27
NORTHERN,ARSON,2,31
NORTHERN,ARSON,3,38
NORTHERN,ARSON,4,27
NORTHERN,ARSON,5,14
NORTHERN,ARSON,6,24
NORTHERN,ARSON,7,11
NORTHERN,ARSON,8,11
NORTHERN,ARSON,9,7


1. Based on above two-line charts, we observed that mission district and Northern district have similar pattern as Southern district with theft dominating the crime category. 
2. Based on the above analysis, theft dominates the crime events in these three districts. In order to decrease the number of crimes, police should put more resource to strike theft crime. 
3. Since the theft crime usually peak during 18 to 19 o'clock. Police department should be more cautious and allocate more resource to prevent the crime during these hours.

In [33]:
#Count the number of crimes in all the district and use a pie chart to help police alocate their resource.
crimeDistrict = spark.sql("SELECT PdDistrict, COUNT(*) AS Count FROM sf_crime GROUP BY 1 ORDER BY Count DESC")
display(crimeDistrict)

PdDistrict,Count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


The above pie chart shows the percentage of the crime number of each district. The police resource should be ditributed accordingly.

#### (OLAP) Analysis of the resolutions of different catogaries

In [36]:
#find out the distribution of crimes in the resolution
temp = spark.sql("SELECT Resolution, count(*) FROM sf_crime GROUP BY Resolution ORDER BY Count(*) DESC")
display(temp)

Resolution,count(1)
NONE,1389500
"ARREST, BOOKED",524979
"ARREST, CITED",154789
LOCATED,34463
PSYCHOPATHIC CASE,29185
UNFOUNDED,23799
JUVENILE BOOKED,14158
COMPLAINANT REFUSES TO PROSECUTE,8089
DISTRICT ATTORNEY REFUSES TO PROSECUTE,7955
NOT PROSECUTED,7720


In [37]:
%sql
 
SELECT category, 
        SUM(case when Resolution ='NONE' then 1 else 0 end)/count(*) as None,
        SUM(case when Resolution ='ARREST, BOOKED' then 1 else 0 end)/count(*) as ARREST_BOOKED,
        SUM(case when Resolution ='ARREST, CITED' then 1 else 0 end)/count(*) as ARREST_CITED,
        SUM(case when Resolution ='LOCATED' then 1 else 0 end)/count(*) as LOCATED,
        SUM(case when Resolution ='PSYCHOPATHIC CASE' then 1 else 0 end)/count(*) as PSYCHOPATHIC,
        SUM(case when Resolution ='UNFOUNDED' then 1 else 0 end)/count(*) as UNFOUNDED,
        SUM(case when Resolution ='JUVENILE BOOKED' then 1 else 0 end)/count(*) as JUVENILE_BOOKED,
        SUM(case when Resolution ='COMPLAINANT REFUSES TO PROSECUTE' then 1 else 0 end)/count(*) as COMPLAINANT_REFUSES_TO_PROSECUTE,
        SUM(case when Resolution ='DISTRICT ATTORNEY REFUSES TO PROSECUTE' then 1 else 0 end)/count(*) as DISTRICT_ATTORNEY_REFUSES_TO_PROSECUTE,
        SUM(case when Resolution ='NOT PROSECUTED' then 1 else 0 end)/count(*) as NO_PROSECUTED,
        SUM(case when Resolution ='JUVENILE CITED' then 1 else 0 end)/count(*) as JUVENILE_CITED,
        SUM(case when Resolution ='PROSECUTED BY OUTSIDE AGENCY' then 1 else 0 end)/count(*) as PROSECUTED_BY_OUTSIDE_AGENCY,
        SUM(case when Resolution ='EXCEPTIONAL CLEARANCE' then 1 else 0 end)/count(*) as EXCEPTIONAL_CLEARANCE,
        SUM(case when Resolution ='JUVENILE ADMONISHED' then 1 else 0 end)/count(*) as JUVENILE_ADMONISHED,
        SUM(case when Resolution ='JUVENILE DIVERTED' then 1 else 0 end)/count(*) as JUVENILE_DIVERTED,
        SUM(case when Resolution ='CLEARED-CONTACT JUVENILE FOR MORE INFO' then 1 else 0 end)/count(*) as CLEARED_CONTACT_JUVENILE_FOR_MORE_INFO,
        SUM(case when Resolution ='PROSECUTED FOR LESSER OFFENSE' then 1 else 0 end)/count(*) as PROSECUTED_FOR_LESSER_OFFENSE
FROM sf_crime 
GROUP BY category 
ORDER BY category DESC
                 


category,None,ARREST_BOOKED,ARREST_CITED,LOCATED,PSYCHOPATHIC,UNFOUNDED,JUVENILE_BOOKED,COMPLAINANT_REFUSES_TO_PROSECUTE,DISTRICT_ATTORNEY_REFUSES_TO_PROSECUTE,NO_PROSECUTED,JUVENILE_CITED,PROSECUTED_BY_OUTSIDE_AGENCY,EXCEPTIONAL_CLEARANCE,JUVENILE_ADMONISHED,JUVENILE_DIVERTED,CLEARED_CONTACT_JUVENILE_FOR_MORE_INFO,PROSECUTED_FOR_LESSER_OFFENSE
WEAPON LAWS,0.2730053071871908,0.5997121525591437,0.052037420167311325,3.148331384366286E-4,8.995232526760817E-4,0.007016281370873437,0.034316812089592516,0.002023927318521184,0.0042277592875775835,0.0029684267338310694,0.01178375461005667,6.746424395070612E-4,0.0017540703427183593,0.00859044706305658,3.148331384366286E-4,3.148331384366286E-4,4.4976162633804086E-5
WARRANTS,0.05407431519348189,0.9182572327602363,0.008729618560056817,3.3537517631856696E-4,3.55103127866718E-4,0.0013020448021779658,0.014766371733791021,2.0714349125558546E-4,3.55103127866718E-4,1.77551563933359E-4,2.56463370125963E-4,4.2415095828524644E-4,2.95919273222265E-4,2.9591927322226496E-5,2.465993943518875E-4,6.904783041852849E-5,1.1836770928890598E-4
VEHICLE THEFT,0.916099271733464,0.03999936809845026,0.0014612723337703985,1.5797538743463768E-4,7.898769371731884E-6,0.037827206521224,0.0018562108023569928,4.186347767017899E-4,5.529138560212319E-4,1.500766180629058E-4,9.715486327230218E-4,8.688646308905073E-5,2.7645692801061595E-4,3.949384685865942E-5,6.319015497385507E-5,2.369630811519565E-5,7.898769371731884E-6
VANDALISM,0.8779155429565997,0.07528929251501391,0.025599048759682577,2.1540768057625863E-4,0.001697412522940918,0.0037653262564730007,0.0025245780163537512,0.002300554028554442,0.0016457146796026159,2.9295444558371175E-4,0.006298520580049803,1.0339568667660414E-4,7.23769806736229E-4,0.0014216906918033069,1.723261444610069E-4,3.446522889220138E-5,0.0
TRESPASS,0.32325569438017376,0.39066275901074604,0.2598077021954856,5.141652527122217E-4,0.0026736593141035527,0.0034963237184431075,0.0037019898195279964,0.0028793254151884416,9.254974548819991E-4,0.0012339966065093321,0.006941230911614993,2.5708262635611086E-4,0.0011311635559668877,0.002210910586662553,1.5424957581366652E-4,1.0283305054244434E-4,5.141652527122217E-5
TREA,0.42857142857142855,0.42857142857142855,0.14285714285714285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SUSPICIOUS OCC,0.8824275272239073,0.03501814927154294,0.011150614091790563,0.0029337178658445625,0.00253592561284869,0.031984983342449404,0.0014917209487345234,0.007719655909701159,0.005046989209885138,0.005767987668440157,7.458604743672617E-4,0.0032196310476853463,0.003430958182089404,0.005208592312664711,4.59947292526478E-4,8.453085376162299E-4,1.2431007906121028E-5
SUICIDE,0.760061919504644,0.02631578947368421,0.0015479876160990713,0.014705882352941176,0.18188854489164086,0.00696594427244582,0.0,0.0,7.739938080495357E-4,0.0,0.0,0.0,0.00696594427244582,0.0,7.739938080495357E-4,0.0,0.0
STOLEN PROPERTY,0.12101589437389622,0.8151543183920612,0.013539651837524178,2.5229164914641325E-4,8.409721638213776E-5,0.0014296526784963418,0.02733159532419477,3.3638886552855103E-4,0.002270624842317719,0.004961735766546127,0.008662013287360189,0.0025229164914641325,0.0012614582457320662,7.568749474392397E-4,1.6819443276427552E-4,1.6819443276427552E-4,8.409721638213776E-5
"SEX OFFENSES, NON FORCIBLE",0.4918793503480278,0.25754060324825984,0.004640371229698376,0.0069605568445475635,0.0,0.030162412993039442,0.02088167053364269,0.058004640371229696,0.09280742459396751,0.009280742459396751,0.0,0.01160092807424594,0.01160092807424594,0.004640371229698376,0.0,0.0,0.0


#### Note(Insight)
1. Based on the above chart, if we sorted the 'None', we will find that for the 'recovered vehicle', 'vehicle theft' and 'larceny/theft', more than 90 percent cases are marked 'None'. That means more than 90 percent of cases have no resolution at all. In order to improve the resolution of the crime related to theft, we recommend the police increase surveillance effort. 
2. From the previous analysis, beside theft, drug related crimes account for a large proportion of the total crime in SF. In this resolution distribution chart, 81 percent of drug crime were resolved by arrest and booked. That means we have a good record of the person who was suspect or prosecuted by drug related charge before on file. In order to decrease the future crime rate in this category, police can focuses on the individuals and groups who drive most of that activity, especially those with the criminal records.

### Clustering

In [40]:
#I want to use the location of crime to do the clustering. So I first pull out the latitude and longitude infromation from the original dataframe.
df_cluster =spark.sql("SELECT IncidntNum as id, X, Y FROM sf_crime")
display(df_cluster)

id,X,Y
146196161,-122.403404791479,37.775420706711
150045675,-122.48560378101,37.7388214326705
140632022,-122.396535107224,37.7106603302503
150383259,-122.400130573297,37.7300925390327
040753980,-120.5,90
040855122,-122.386667033903,37.7898821569191
066085191,-122.38750147945,37.716878646429
050908404,-120.5,90
090768064,-122.399686082806,37.739901780585
111027676,-122.412933062384,37.7739274524819


In [41]:
#Becuase the input of kmeans package should be feature vectors with float type. We need to transform the data frame.
FEATURES_COL = ['X', 'Y']
#df_cluster = df_cluster.select(*(df_cluster[c].cast("float").alias(c) for c in df_cluster.columns[1:]))
#df_cluster.show()
for col in df_cluster.columns:
    if col in FEATURES_COL:
        df_cluster = df_cluster.withColumn(col,df_cluster[col].cast('float'))
df_cluster.show()



+---------+-----------+---------+
 id| X| Y|
+---------+-----------+---------+
146196161|-122.403404| 37.77542|
150045675| -122.4856|37.738823|
140632022| -122.39654| 37.71066|
150383259| -122.40013| 37.73009|
040753980| -120.5| 90.0|
040855122|-122.386665|37.789883|
066085191|-122.387505|37.716877|
050908404| -120.5| 90.0|
090768064| -122.39969|37.739902|
111027676| -122.41293|37.773926|
060508925| -122.41984|37.786358|
110318854| -122.39895| 37.73385|
131007921| -122.43571| 37.74437|
090333811| -122.42801|37.772472|
050951798| -120.5| 90.0|
031057056|-122.386665|37.789883|
061079105|-122.428314|37.782364|
050326981| -122.39525|37.755344|
120444392|-122.458725| 37.78028|
051130171| -120.5| 90.0|
+---------+-----------+---------+
only showing top 20 rows

In [42]:
#Transform data fram into vector sets
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
df_kmeans = vecAssembler.transform(df_cluster).select('id', 'features')
df_kmeans.show()

+---------+--------------------+
 id| features|
+---------+--------------------+
146196161|[-122.40340423583...|
150045675|[-122.48560333251...|
140632022|[-122.39653778076...|
150383259|[-122.40013122558...|
040753980| [-120.5,90.0]|
040855122|[-122.38666534423...|
066085191|[-122.38750457763...|
050908404| [-120.5,90.0]|
090768064|[-122.39968872070...|
111027676|[-122.41293334960...|
060508925|[-122.41983795166...|
110318854|[-122.39894866943...|
131007921|[-122.43570709228...|
090333811|[-122.42800903320...|
050951798| [-120.5,90.0]|
031057056|[-122.38666534423...|
061079105|[-122.42831420898...|
050326981|[-122.39524841308...|
120444392|[-122.45872497558...|
051130171| [-120.5,90.0]|
+---------+--------------------+
only showing top 20 rows

In [43]:

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator



# Trains a k-means model.
kmeans = KMeans().setK(5).setSeed(1)
model = kmeans.fit(df_kmeans)

# Make predictions
predictions = model.transform(df_kmeans)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)




Silhouette with squared euclidean distance = 0.5597636034987195
Cluster Centers: 
[-122.47652315 37.74754932]
[-120.5 90. ]
[-122.40918075 37.73509192]
[-122.40945435 37.78490068]
[-122.43321491 37.77299508]

In [44]:
predictions.show()

+---------+--------------------+----------+
 id| features|prediction|
+---------+--------------------+----------+
146196161|[-122.40340423583...| 3|
150045675|[-122.48560333251...| 0|
140632022|[-122.39653778076...| 2|
150383259|[-122.40013122558...| 2|
040753980| [-120.5,90.0]| 1|
040855122|[-122.38666534423...| 3|
066085191|[-122.38750457763...| 2|
050908404| [-120.5,90.0]| 1|
090768064|[-122.39968872070...| 2|
111027676|[-122.41293334960...| 3|
060508925|[-122.41983795166...| 3|
110318854|[-122.39894866943...| 2|
131007921|[-122.43570709228...| 2|
090333811|[-122.42800903320...| 4|
050951798| [-120.5,90.0]| 1|
031057056|[-122.38666534423...| 3|
061079105|[-122.42831420898...| 4|
050326981|[-122.39524841308...| 2|
120444392|[-122.45872497558...| 4|
051130171| [-120.5,90.0]| 1|
+---------+--------------------+----------+
only showing top 20 rows

In [45]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  
import numpy as np

#### Note(Insight)
Based on the above K means model, we located five central point of crime events. These peice information can be used to alocate police resource. For example, in the daily peak of crime, gorverment can deploy more police to patrol around these loacations to prevent crime.

### Conclusion. 

Point 1:  San Francisco as one of the largest cities on the West Coast, is the cultural, commercial, and financial center. Everyday, millions of arties, businessman and travelers come to the city. The most import question in their mind is whether San Francisco is safe or not. My analysis is to better understand the crime pattern in San Francisco so that we can have a more constructive advice to our travelers and to our policy makers.  
Point 2:  To accomplish this goal, we used the Spark SQL and Spark Data frame to analysis the data. First we use data visualization and some data manipulation to explore the patterns from different prospects. Then we used k means clustering to study the data.  
Point 3:  The data has obvious yearly and daily patterns. And the crime tends to cluster in certain area. We suggest travelers to travels in a lower crime rate time and avoid the dangerous area. And police can plan the deployment based on the grime trends during recent years and based on the distribution of the different crimes.